In [ ]:
import os
import time
import google.generativeai as genai

# ---------------------------------------------------------
# 1. API 설정
# ---------------------------------------------------------
# Google AI Studio에서 생성한 API 키 입력
API_KEY = "" 
genai.configure(api_key=API_KEY)

# 모델 설정
model = genai.GenerativeModel('gemini-flash-latest')

# ---------------------------------------------------------
# 2. 파일 읽기 함수 (단일 파일)
# ---------------------------------------------------------
def read_text_from_file(file_path):
    """지정된 경로의 파일 내용을 읽어서 문자열로 반환"""
    if not os.path.exists(file_path):
        print(f"⚠️ 파일 없음: {file_path}")
        return None
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"⚠️ 파일 읽기 오류 ({file_path}): {e}")
        return None

# ---------------------------------------------------------
# 3. LLM 키워드 추출 함수
# ---------------------------------------------------------
def extract_vibe_with_llm(team_name, text_data):
    if not text_data:
        print(f"❌ {team_name}: 분석할 데이터가 없습니다.")
        return "데이터 없음"

    # 입력 길이 제한 (비용 및 쿼터 관리)
    input_text = text_data[:20000] # 분석에 충분하도록 2만 자 정도 사용
    
    print(f"▶ [{team_name}] 분석 요청 중... (전송 데이터: {len(input_text)}자)")

    prompt = f"""
    너는 스포츠 데이터 분석 전문가야. 아래 텍스트는 F1(포뮬러 1) 팀 '{team_name}'에 대한 데이터야.
    이 팀의 **고유한 성향, 레이싱 스타일, 팬덤 분위기**를 나타내는 핵심 키워드 15개를 한글로 번역해서 추출해줘.
    
    [조건]
    1. 'F1', '레이싱', '자동차', '우승', '드라이버' 같은 뻔한 단어 제외.
    2. 형용사나 명사 위주 (예: 혁신적인, 공격적인 전략, 귀족적인, 압도적인).
    3. 결과는 오직 키워드만 쉼표(,)로 구분해서 출력해.
    
    [텍스트]
    {input_text}
    """

    try:
        response = model.generate_content(prompt)
        if response.text:
            return response.text.strip()
        else:
            return "응답 없음"
    except Exception as e:
        print(f"⚠️ API 호출 에러: {e}")
        return "Error"

# ---------------------------------------------------------
# 4. 실행 데이터 설정
# ---------------------------------------------------------
# 딕셔너리 구조 변경: { "팀이름": "파일경로" }
teams = {
    "Jeju Test": r"./Crawled_Data/jeju_utd.txt",
    "Jeju Clean": r"./Crawled_Data/jeju_utd_cleaned.txt"
}

# ---------------------------------------------------------
# 5. 메인 루프 실행
# ---------------------------------------------------------
results = {}

print("🚀 LLM F1 팀 성향 분석 시작 (단일 파일 모드)...\n")

for team_name, file_path in teams.items():
    
    # 1. 파일에서 텍스트 읽기
    text_data = read_text_from_file(file_path)
    
    # 2. 데이터가 있을 경우에만 분석 진행
    if text_data:
        keywords = extract_vibe_with_llm(team_name, text_data)
        results[team_name] = keywords
        
        print(f"=== 📊 {team_name} 결과 ===")
        print(keywords)
        print("-" * 50 + "\n")
        
        # 속도 조절
        print("⏳ API 과부하 방지를 위해 5초 대기 중...")
        time.sleep(5)
    else:
        print(f"pass: {team_name} (파일을 읽지 못함)\n")

c:\Project\2025_Sports_Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 LLM F1 팀 성향 분석 시작 (단일 파일 모드)...

▶ [Jeju Test] 분석 요청 중... (전송 데이터: 20000자)
⚠️ API 호출 에러: 403 Your API key was reported as leaked. Please use another API key.
=== 📊 Jeju Test 결과 ===
Error
--------------------------------------------------

⏳ API 과부하 방지를 위해 5초 대기 중...
▶ [Jeju Clean] 분석 요청 중... (전송 데이터: 20000자)
⚠️ API 호출 에러: 403 Your API key was reported as leaked. Please use another API key.
=== 📊 Jeju Clean 결과 ===
Error
--------------------------------------------------

⏳ API 과부하 방지를 위해 5초 대기 중...
